In [2]:
from dataclasses import dataclass
from pathlib import Path
import os
import gdown
import zipfile
from classifier import logger
from classifier.utils.common import get_size

In [12]:
os.chdir('..')

'c:\\Users\\Kushal\\Documents\\GitHub\\kidney-disease-mlops'

In [13]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
from classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from classifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir = config.root_dir,
                                                    source_url= config.source_url,
                                                    local_data_file= config.local_data_file,
                                                    unzip_dir= config.unzip_dir)
        
        return data_ingestion_config

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir , exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file {zip_download_dir}')

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Data downloaded from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e 
    
    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info('Zip file extracted')

In [19]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()

except Exception as e:
    raise e

[2024-07-30 13:36:56,293: INFO: common yaml file: config\config.yaml loaded successfully]
[2024-07-30 13:36:56,297: INFO: common yaml file: params.yaml loaded successfully]
[2024-07-30 13:36:56,299: INFO: common created directory at: artifacts]
[2024-07-30 13:36:56,301: INFO: common created directory at: artifacts/data_ingestion]
[2024-07-30 13:36:56,303: INFO: 1401358270 Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=9eebee00-1ebd-4faf-9b8e-1321b132ed04
To: c:\Users\Kushal\Documents\GitHub\kidney-disease-mlops\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:04<00:00, 12.3MB/s]

[2024-07-30 13:37:04,828: INFO: 1401358270 Data downloaded from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view into file artifacts/data_ingestion/data.zip]
